<a href="https://colab.research.google.com/github/jyryu3161/biocomputing/blob/main/lab9_%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%80%E1%85%B2%E1%84%91%E1%85%AD%E1%84%92%E1%85%A7%E1%86%AB%E1%84%89%E1%85%B5%E1%86%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 정규 표현식의 기초, 메타 문자
메타 문자(meta characters)란, 문자 자체가 아니라 **특별한 의미**를 갖도록 약속된 문자입니다.

`. ^ $ * + ? { } [ ] \ | ( )`

In [1]:
# 메타 문자를 간단히 확인해봅시다 (문자열로 출력)
meta_chars = ". ^ $ * + ? { } [ ] \\ | ( )"
print("메타 문자:", meta_chars)

메타 문자: . ^ $ * + ? { } [ ] \ | ( )


## `[ ]` 문자 — 문자 클래스 (character class)
문자 클래스는 *"이 중에서 아무거나 하나"*라는 의미입니다. 예: `[abc]` → `'a'` 또는 `'b'` 또는 `'c'` 중 하나.

**범위**는 하이픈 `-`으로 표시합니다. 예: `[a-z]`, `[0-9]`, `[가-힣]`.

**제외**는 맨 앞에 `^`를 둡니다. 예: `[^0-9]` → 숫자가 아닌 모든 문자.

In [2]:
import re

samples = ["a", "before", "dude", "Z", "가"]
pattern = re.compile(r"[abc]")

for s in samples:
    m = pattern.search(s)
    print(f"{s!r:>8} ->", "매치됨" if m else "매치되지 않음")

     'a' -> 매치됨
'before' -> 매치됨
  'dude' -> 매치되지 않음
     'Z' -> 매치되지 않음
     '가' -> 매치되지 않음


### 범위 지정 예시

In [3]:
import re

tests = ["A", "G", "z", "5", "한", "힣", "가"]
p1 = re.compile(r"[A-Z]")
p2 = re.compile(r"[a-zA-Z]")
p3 = re.compile(r"[0-9]")
p4 = re.compile(r"[가-힣]")

for t in tests:
    print(t,
          " A-Z:", bool(p1.search(t)),
          " a-zA-Z:", bool(p2.search(t)),
          " 0-9:", bool(p3.search(t)),
          " 가-힣:", bool(p4.search(t)))

A  A-Z: True  a-zA-Z: True  0-9: False  가-힣: False
G  A-Z: True  a-zA-Z: True  0-9: False  가-힣: False
z  A-Z: False  a-zA-Z: True  0-9: False  가-힣: False
5  A-Z: False  a-zA-Z: False  0-9: True  가-힣: False
한  A-Z: False  a-zA-Z: False  0-9: False  가-힣: True
힣  A-Z: False  a-zA-Z: False  0-9: False  가-힣: True
가  A-Z: False  a-zA-Z: False  0-9: False  가-힣: True


### 자주 사용하는 축약 클래스
`\d`(숫자), `\D`(숫자 아님), `\s`(공백), `\S`(공백 아님), `\w`(문자+숫자+언더스코어), `\W`(그 외)

In [ ]:
import re

text = "ID: user_01\tScore: 98\nNext: 100"
print("원문:", repr(text))

print("\\d 모든 숫자:", re.findall(r"\d", text))
print("\\s 모든 공백:", re.findall(r"\s", text))
print("\\w 모든 문자/숫자/_:", re.findall(r"\w", text))

r"\d"에서 r은 **raw string(원시 문자열)**을 의미
즉, r"..." 형태로 문자열을 작성하면, 파이썬이 역슬래시(\)를 특수문자로 해석하지 않고 그대로 사용하도록 하는 표시

## `.` (dot) — `\n`을 제외한 모든 문자
`.`은 줄바꿈(`\n`)을 제외한 **아무 문자 1개**를 의미합니다.

In [ ]:
import re
p = re.compile(r"a.b")   # 'a' + 아무문자 1개 + 'b'
for s in ["aab", "a0b", "abc"]:
    print(s, "->", bool(p.fullmatch(s)))

> `[]` 안에 `.`을 쓰면 리터럴 점(마침표) 자체를 의미합니다. 예: `a[.]b`는 `'a.b'`와만 매치.

In [ ]:
import re
print(bool(re.fullmatch(r"a[.]b", "a.b")), bool(re.fullmatch(r"a[.]b", "a0b")))

## 반복 메타 문자: `*`, `+`, `?`, `{m,n}`
- `*` : 0회 이상 반복
- `+` : 1회 이상 반복
- `?` : 0회 또는 1회 (있어도 되고 없어도 됨)
- `{m,n}` : m회 이상 n회 이하 반복

### `*` 예시 (`ca*t`)

In [4]:
import re
p = re.compile(r"ca*t")
tests = ["ct", "cat", "caaat"]
for t in tests:
    print(t, "->", bool(p.fullmatch(t)))

ct -> True
cat -> True
caaat -> True


### `+` 예시 (`ca+t`)

In [5]:
import re
p = re.compile(r"ca+t")
tests = ["ct", "cat", "caaat"]
for t in tests:
    print(t, "->", bool(p.fullmatch(t)))

ct -> False
cat -> True
caaat -> True


### `{m}` / `{m,n}` 예시

In [6]:
import re
print("ca{2}t:", bool(re.fullmatch(r"ca{2}t", "caat")), bool(re.fullmatch(r"ca{2}t", "cat")))
print("ca{2,5}t:", [bool(re.fullmatch(r"ca{2,5}t", s)) for s in ["cat","caat","caaaaat"]])

ca{2}t: True False
ca{2,5}t: [False, True, True]


### `?` 예시 (`ab?c`)

In [7]:
import re
p = re.compile(r"ab?c")
tests = ["abc", "ac", "abbc"]
for t in tests:
    print(t, "->", bool(p.fullmatch(t)))

abc -> True
ac -> True
abbc -> False


### 소괄호 () — 그룹화 (grouping)

()는 정규식에서 하위 패턴(subpattern)을 묶는 데 사용.
이렇게 묶은 그룹은 하나의 단위처럼 동작하고, match.group(n)으로 부분 문자열을 참조

In [ ]:
import re

pattern = re.compile(r"(\d{3})-(\d{4})-(\d{4})")
m = pattern.search("전화번호: 010-1234-5678")

if m:
    print("전체:", m.group(0))
    print("국번:", m.group(1))
    print("앞자리:", m.group(2))
    print("뒷자리:", m.group(3))

## 파이썬에서 정규 표현식을 지원하는 `re` 모듈
`re.compile(pattern)`로 **패턴 객체**를 만들고, 이를 재사용하여 검색합니다.

In [8]:
import re
p = re.compile(r"[a-z]+")
print("패턴 객체:", p)
print("match('python'):", p.match("python"))
print("match('3 python'):", p.match("3 python"))

패턴 객체: re.compile('[a-z]+')
match('python'): <re.Match object; span=(0, 6), match='python'>
match('3 python'): None


## 정규식을 이용한 문자열 검색 — `match`, `search`, `findall`, `finditer`

### `match`: 문자열 **처음부터** 매치 여부를 확인

In [ ]:
import re
p = re.compile(r"[a-z]+")
print("python ->", p.match("python"))
print("3 python ->", p.match("3 python"))

### `search`: 문자열 **전체**에서 첫 매치를 찾음

In [ ]:
import re
p = re.compile(r"[a-z]+")
print("python ->", p.search("python"))
print("3 python ->", p.search("3 python"))

### `findall`: 모든 매치 결과를 **리스트**로 반환

In [ ]:
import re
p = re.compile(r"[a-z]+")
print(p.findall("life is too short"))

### `finditer`: 모든 매치 결과를 **이터레이터**(각 원소는 Match 객체)로 반환

In [ ]:
import re
p = re.compile(r"[a-z]+")
for m in p.finditer("life is too short"):
    print(m, "->", m.group(), m.span())

## 역슬래시(\) 문제와 raw string `r''`
파이썬 문자열 리터럴에서 `\`는 이스케이프 문자로 쓰입니다. 정규식에도 `\d`, `\s`처럼 `\`가 쓰이므로, 혼동을 피하려면 **raw string**을 권장합니다.

In [10]:
import re

# 의도: 문자열 안에서 "\section"을 찾기
target = r"\section is here"

# 잘못된 패턴: \s 가 '공백'으로 해석되어버림
wrong = re.compile("\\section")
print("잘못된 패턴 매치:", bool(wrong.search(target)))

# 올바른 패턴 1: 역슬래시를 이스케이프
ok1 = re.compile("\\\\section")
print("올바른 패턴(이스케이프) 매치:", bool(ok1.search(target)))

# 올바른 패턴 2: raw string 사용
ok2 = re.compile(r"\\section")
print("올바른 패턴(raw) 매치:", bool(ok2.search(target)))

잘못된 패턴 매치: False
올바른 패턴(이스케이프) 매치: True
올바른 패턴(raw) 매치: True


## 실습1 — 이메일 추출

In [11]:
import re

text = """
연락처 목록
- 영업팀: sales-team@myco.co
- 개발팀: dev@myco.io
- 잘못된: name@bad@site
- 개인: first.last@university.ac.kr
"""

pattern = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
print(pattern.findall(text))

['sales-team@myco.co', 'dev@myco.io', 'first.last@university.ac.kr']


# 실습2- DNA 서열에서 특정 염기 패턴 찾기

DNA에서 ATG로 시작해서 TAA, TAG, TGA 중 하나로 끝나는 open reading frame (ORF) 패턴을 찾는 예제

In [12]:
import re

dna = "AATGCGATGCCCTAGATGAAATGCGTTGAATGCCCTAA"
pattern = re.compile(r"ATG(?:[ATGC]{3})*?(?:TAA|TAG|TGA)")

matches = pattern.findall(dna)
print("DNA에서 ORF로 추정되는 서열:")
for m in matches:
    print("-", m)

DNA에서 ORF로 추정되는 서열:
- ATGCGATGCCCTAGATGA
- ATGCGTTGA
- ATGCCCTAA


- 의미: 그룹을 묶긴 하지만, 나중에 group()으로 캡처하지 않음.
- 용도: 반복 제어나 | (or) 선택을 할 때, 불필요한 그룹 인덱싱을 피할 때.
- ()는 그룹 번호를 만들어서 group(1)처럼 쓸 수 있지만,
- (?:...)는 그룹으로 묶되 번호를 매기지 않음.